In [1]:
import numpy as np
from keras import Input
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from skvideo.measure import niqe, viideo_score, viideo_features, videobliinds_features, brisque_features
from skvideo.motion import globalEdgeMotion, blockMotion

import helper_functions as hf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, \
    Dropout, Conv2D, MaxPooling2D, TimeDistributed, LSTM, LeakyReLU, Average, Lambda, K, Conv3D, MaxPooling3D, \
    GlobalMaxPooling3D, BatchNormalization, GlobalAveragePooling3D
from keras.utils import to_categorical
from imgaug import augmenters as iaa

Using TensorFlow backend.


In [2]:
def read_data():
    X_train = hf.import_video_data(158, "task4/input/train/")
    X_test = hf.import_video_data(69, "task4/input/test/")

    test_index = np.arange(69)

    y_train, _ = hf.read_csv_to_matrix("task4/input/train_target.csv", "id")
    return X_train, X_test, np.squeeze(y_train), test_index

In [3]:
X_train, X_test, y_train, test_index = read_data()

In [4]:
def preprocessing(X_train, X_test, y_train):
    X_train_new = []
    X_test_new = []
    y_train_new = []
    train_frame_index = [0]
    test_frame_index = [0]

    for sample_idx in range(X_train.shape[0]):
        sample = X_train[sample_idx]
        train_frame_index.append(train_frame_index[sample_idx] + sample.shape[0])

        for frame_idx in range(sample.shape[0]):
            frame = sample[frame_idx]
            X_train_new.append(frame)
            y_train_new.append(y_train[sample_idx])

    for sample_idx in range(X_test.shape[0]):
        sample = X_test[sample_idx]
        test_frame_index.append(test_frame_index[sample_idx] + sample.shape[0])
        
        for frame_idx in range(sample.shape[0]):
            frame = sample[frame_idx]
            X_test_new.append(frame)

    return np.asarray(X_train_new), np.asarray(X_test_new), np.asarray(y_train_new), np.asarray(
        train_frame_index), np.asarray(test_frame_index)


In [44]:
def feature_extraction(X_train, y_train):
    X_train_new = []
    y_train_new = []

    aug = iaa.Sequential([
        iaa.Affine(scale=(0.75, 1.25), rotate=(-10, 10), translate_px=(-10, 10)),
        iaa.ContrastNormalization((0.5, 1.5))
        ], random_order=True)

    for sample_idx in range(X_train.shape[0]):
        # print(str(sample_idx) + "/" + str(X_train.shape[0] - 1))

        sample_X = X_train[sample_idx]
        sample_y = y_train[sample_idx]
        """
        vectorfields = blockMotion(videodata=sample_X, method='DS', mbSize=10, p=2)
        mean = np.mean(vectorfields, axis=0)
        std = np.std(vectorfields, axis=0)

        X_train_new.append(np.r_[mean.flatten(), std.flatten()])
        """
        X_train_new.append(sample_X)
        y_train_new.append(sample_y)
        
        # make augmentation
        for i in range(10):
            aug_det = aug.to_deterministic()
            sample_aug = aug_det.augment_images(sample_X)
            """
            vectorfields = blockMotion(videodata=sample_aug, method='DS', mbSize=10, p=2)
            mean = np.mean(vectorfields, axis=0)
            std = np.std(vectorfields, axis=0)

            X_train_new.append(np.r_[mean.flatten(), std.flatten()])
            """
            X_train_new.append(sample_aug)
            y_train_new.append(sample_y)
 
    return np.asarray(X_train_new), np.asarray(y_train_new)

In [45]:
X_train_new, y_train_new = feature_extraction(X_train, y_train)

In [46]:
X_train_new, X_test_new, y_train_new, train_frame_index, test_frame_index = preprocessing(X_train_new, X_test, y_train_new)

In [47]:
np.shape(X_train_new)

In [51]:
from keras.layers import GlobalMaxPooling2D


def cnn():
    model = Sequential()
    model.add(Conv2D(filters=4, kernel_size=(5, 5), use_bias=False, input_shape=(100, 100, 1)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(filters=8, kernel_size=(4, 4), use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(filters=16, kernel_size=(3, 3), use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(filters=64, kernel_size=(2, 2), use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(MaxPooling2D((2, 2)))
    
    #model.add(GlobalMaxPooling2D())
    model.add(Flatten())
    
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.5))
    
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.5))
    
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.5))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [52]:
clf = KerasClassifier(build_fn=cnn, epochs=2, batch_size=100, verbose=1)

In [ ]:

print("==> Crossvalidation")
results = cross_val_score(estimator=clf, X=X_train_new, y=y_train_new, cv=5, n_jobs=1, scoring=hf.scorer(),
                          pre_dispatch='n_jobs', verbose=1)
print("Results: %.4f (%.4f) MSE" % (results.mean(), results.std()))


In [53]:
clf.fit(X_train_new, y_train_new)
y_pred = clf.predict_proba(X_test_new)
y_pred_new = []
for idx in range(test_frame_index.shape[0]-1):
    y_pred_new.append(np.mean(y_pred[test_frame_index[idx]: test_frame_index[idx+1], 1]))

print("==> Printing solution")
hf.write_to_csv_from_vector("task4/output/solution_franz_1.csv", test_index, np.asarray(y_pred_new), "id")

Epoch 1/2


   100/113410 [..............................] - ETA: 1:13:47 - loss: 0.7986 - acc: 0.5100

   200/113410 [..............................] - ETA: 45:51 - loss: 0.8530 - acc: 0.4950  

   300/113410 [..............................] - ETA: 36:45 - loss: 0.8226 - acc: 0.5167

   400/113410 [..............................] - ETA: 32:03 - loss: 0.8375 - acc: 0.5225

...

4400/4665 [===========================>..] - ETA: 0s

4500/4665 [===========================>..] - ETA: 0s

4600/4665 [============================>.] - ETA: 0s

4665/4665 [==============================] - 13s 3ms/step


==> Printing solution
